## UAV fleet Simulation 

This is for testing propouses

In [1]:
# Custom modules
from config import Color
from helpers import clean  #, local2global
from helpers.change_coordinates import ENUs_to_GRAs, rel_to_abs
from mavlink.customtypes.location import ENU, ENUPose, GRAPose
from mavlink.util import save_mission
from plan import Plan
from simulators import QGC, ConfigGazebo, Gazebo, Simulator

clean()

In [2]:
# offsets= [  # east, north, up, heading
#     ENUPose(0., 0., 0., 0.),
#     ENUPose(10., 0., 0., 45.),
#     ENUPose(-5., -10., 0., 225.),
#     ENUPose(-15., 0., 0., 0.),
#     ENUPose(0., -20., 0., 0.),
# ]
# origin=GRAPose(-35.3633245, 149.1652241, 0, 0)


# n_vehicles = len(offsets)
# side_lens = (5, 5, 5, 5, 5)  # must agree with n_vehicles
# local_paths = [
#     Plan.create_square_path(side_len=side_len, alt=5, clockwise=True)
#     for side_len in side_lens
# ]

# GRAoffsets = ENUs_to_GRAs(origin, offsets)
# GRApaths = [
#     ENUs_to_GRAs(offset, loc_path)
#     for offset, loc_path in zip(GRAoffsets, local_paths)
# ]


## Create mission

In [3]:
mission_name = "square"
offset = ENUPose(-1,1,0,0)  # east, north, up, heading 
origin = GRAPose(-35.3633245, 149.1652241, 0, 0)
rel_path = Plan.create_square_path(side_len=5, alt=5)

## Convert to Global (ENU) coordinates

In [4]:
home = ENU(*offset[:3])
abs_path = [rel_to_abs(pos, home, cls=ENU) for pos in rel_path]

## Create Plan

In [5]:
gra_offset = ENUs_to_GRAs(origin, [offset])[0]
gra_path = ENUs_to_GRAs(origin, abs_path)
save_mission(name=mission_name,poses=gra_path)
plan = Plan.auto(name="square_auto", mission_name="square",check_until=len(abs_path)+3)

## Visualization Parameters

In [6]:
home = ENU(*offset[:3])
color = Color.BLUE
markers = [
	ConfigGazebo.create_markertraj(traj=abs_path, color=color)
]
Gazebo.plot_3d_interactive(
	markers=markers,
	title="Simulation Waypoints",
	frames=(0.2, 0.2, 0.6),
	ground=-0.05
)

## Choose Simulator

In [ ]:
simulator = Simulator(offsets=[offset], plans=[plan],verbose=3)

simulator = QGC(offsets=[offset], plans=[plan], origin=origin)

gazebo_config = ConfigGazebo(world_path="simulators/gazebo/worlds/runway.world",
                           models=[("iris",color)],
                           marker_trajs = markers)
simulator = Gazebo(offsets=[offset],plans=[plan],config=gazebo_config)



TypeError: Gazebo.__init__() got an unexpected keyword argument 'verbose'

Launch Simulator

In [ ]:
orac = simulator.launch(gcs_sysids={'blue 🟦':[1]})

🖥️ Gazebo launched for realistic simulation and 3D visualization.
🚀 ArduPilot SITL vehicle 1 launched (PID 2463722)
🚀 UAV logic for vehicle 1 launched (PID 2463724)
🚀 Proxy for vehicle 1 launched (PID 2463726)
🔗 UAV logic 1 is connected to Ardupilot SITL vehicle 1
System id: 1
[Errno 111] Connection refused sleeping
System id: 1
SIM_VEHICLE: Start
SIM_VEHICLE: Killing tasks
SIM_VEHICLE: Starting up at SITL location
SIM_VEHICLE: Using defaults from (../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm)
SIM_VEHICLE: Adding parameters from (/home/abeldg/uav-cyber-sim/params/vehicle.parm)
Setting SIM_SPEEDUP=1.000000
Setting MAV_SYSID=1
Starting SITL Gazebo
Bind 127.0.0.1:9003 for SITL in
Setting Gazebo interface to 127.0.0.1:9002 
Starting sketch 'ArduCopter'
Starting SITL input
Using Irlock at port : 9005
Waiting for connection ....


bind port 5760 for SERIAL0
SERIAL0 on TCP port 5760


Connection on serial port 5760

🚀 Starting Proxy 1
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Home: -35.363262 149.165237 alt=584.000000m hdg=353.000000


Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
Warning [parser.cc:833] XML Attribute[version] in element[sdf] not defined in SDF, ignoring.
bind port 5762 for SERIAL1
SERIAL1 on TCP port 5762
bind port 5763 for SERIAL2
SERIAL2 on TCP port 5763
validate_structures:528: Validating structures


Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/default_params/copter.parm,../../ardupilot/Tools/autotest/default_params/gazebo-iris.parm,/home/abeldg/uav-cyber-sim/params/vehicle.parm
Loaded defaults from ../../ardupilot/Tools/autotest/def

## Execute Plan

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

Vehicle 1: ▶️ Action Started: 📤 UPLOAD_MISSION
Vehicle 1: ▶️ Step Started: clear uav missions
SIM_VEHICLE: Waiting for SITL to exit
🔗 UAV logic 1 is connected
Vehicle 1: ✅ Step Done: clear uav missions
✅ 7 waypoints read.
✅ Sent mission item 0
✅ Sent mission item 1
✅ Sent mission item 2
✅ Sent mission item 3
✅ Sent mission item 4
✅ Sent mission item 5
✅ Sent mission item 6
Vehicle 1: ▶️ Step Started: load mission square
🎉 Mission upload successful!
Vehicle 1: ✅ Step Done: load mission square
Vehicle 1: ✅ Action Done: 📤 UPLOAD_MISSION
Vehicle 1: ▶️ Action Started: 🔧 PREARM
Vehicle 1: ▶️ Step Started: Check disarmed
Vehicle 1: ✅ Step Done: Check disarmed
Vehicle 1: ▶️ Step Started: Check EKF status
Vehicle 1: ✅ Step Done: Check EKF status
Vehicle 1: ▶️ Step Started: Check GPS
Vehicle 1: ✅ Step Done: Check GPS
Vehicle 1: ▶️ Step Started: Check system
Vehicle 1: ✅ Step Done: Check system
Vehicle 1: ✅ Action Done: 🔧 PREARM
Vehicle 1: ▶️ Action Started: ⚙️ MODE: GUIDED
Vehicle 1: ▶️ Step Sta

📤 GCS ← UAV 1: Sending DONE (attempt 2)
📤 GCS ← UAV 1: Sending DONE (attempt 3)
📤 GCS ← UAV 1: Sending DONE (attempt 4)
📤 GCS ← UAV 1: Sending DONE (attempt 5)
📤 GCS ← UAV 1: Sending DONE (attempt 6)
📤 GCS ← UAV 1: Sending DONE (attempt 7)
📤 GCS ← UAV 1: Sending DONE (attempt 8)
📤 GCS ← UAV 1: Sending DONE (attempt 9)
📤 GCS ← UAV 1: Sending DONE (attempt 10)
📤 GCS ← UAV 1: Sending DONE (attempt 11)
✅ Vehicle 1 completed its mission
✅ All UAVs assigned to GCS blue 🟦 have completed their mission.
💾 Trajectories saved to 'trajectories_blue 🟦.pkl'.
✅ ACK received. DONE message acknowledged.
❎ Vehicle 1 logic stopped.
Connection reset or closed by peer on TCP socket
❎ Proxy 1 stopped.
Closed connection on SERIAL0
